In [4]:
import sys

!{sys.executable} -m pip install --upgrade numpy --user --no-warn-script-location
!{sys.executable} -m pip install tblib


Requirement already up-to-date: numpy in c:\users\stsc\appdata\roaming\python\python37\site-packages (1.17.4)


In [57]:
import tensorflow as tf
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tqdm
import datetime
import json
from sklearn.preprocessing import  LabelBinarizer
import pickle
import import_ipynb
import LSTM_XX_Training

In [2]:
# enable eager execution to guarantee the tensors have discrete instead of symbolic values, 
# so you can do operations on them
tf.enable_eager_execution()
tf.executing_eagerly()

True

In [3]:
# create MobileNetV2 feature extraction model
mobilenet_v2 = tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape=(224,224,3), include_top=False, weights='imagenet')
x = mobilenet_v2.output

pooling_output = tf.keras.layers.GlobalAveragePooling2D()(x)
feature_extraction_model = tf.keras.Model(mobilenet_v2.input,pooling_output)

In [4]:
feature_extraction_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
______________________________________________________________________________________________

block_4_expand_relu (ReLU)      (None, 28, 28, 192)  0           block_4_expand_BN[0][0]          
__________________________________________________________________________________________________
block_4_depthwise (DepthwiseCon (None, 28, 28, 192)  1728        block_4_expand_relu[0][0]        
__________________________________________________________________________________________________
block_4_depthwise_BN (BatchNorm (None, 28, 28, 192)  768         block_4_depthwise[0][0]          
__________________________________________________________________________________________________
block_4_depthwise_relu (ReLU)   (None, 28, 28, 192)  0           block_4_depthwise_BN[0][0]       
__________________________________________________________________________________________________
block_4_project (Conv2D)        (None, 28, 28, 32)   6144        block_4_depthwise_relu[0][0]     
__________________________________________________________________________________________________
block_4_pr

__________________________________________________________________________________________________
block_8_project (Conv2D)        (None, 14, 14, 64)   24576       block_8_depthwise_relu[0][0]     
__________________________________________________________________________________________________
block_8_project_BN (BatchNormal (None, 14, 14, 64)   256         block_8_project[0][0]            
__________________________________________________________________________________________________
block_8_add (Add)               (None, 14, 14, 64)   0           block_7_add[0][0]                
                                                                 block_8_project_BN[0][0]         
__________________________________________________________________________________________________
block_9_expand (Conv2D)         (None, 14, 14, 384)  24576       block_8_add[0][0]                
__________________________________________________________________________________________________
block_9_ex

__________________________________________________________________________________________________
block_13_expand_BN (BatchNormal (None, 14, 14, 576)  2304        block_13_expand[0][0]            
__________________________________________________________________________________________________
block_13_expand_relu (ReLU)     (None, 14, 14, 576)  0           block_13_expand_BN[0][0]         
__________________________________________________________________________________________________
block_13_pad (ZeroPadding2D)    (None, 15, 15, 576)  0           block_13_expand_relu[0][0]       
__________________________________________________________________________________________________
block_13_depthwise (DepthwiseCo (None, 7, 7, 576)    5184        block_13_pad[0][0]               
__________________________________________________________________________________________________
block_13_depthwise_BN (BatchNor (None, 7, 7, 576)    2304        block_13_depthwise[0][0]         
__________

In [5]:
# save CNN model in SavedModel format for converting to tflite
SAVED_MODEL_DIR = 'C:/Users/STSC/Desktop/Project3/SavedModelDir/LSTM/v1'
tf.keras.experimental.export_saved_model(feature_extraction_model, SAVED_MODEL_DIR)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: No

In [6]:
# convert the SavedModel to a TFLite model
converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL_DIR)
tflite_model = converter.convert()



Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:/Users/STSC/Desktop/Project3/SavedModelDir/LSTM/v1\variables\variables
INFO:tensorflow:The given SavedModel MetaGraphDef contains SignatureDefs with the following keys: {'__saved_model_init_op', 'serving_default'}
INFO:tensorflow:input tensors info: 
INFO:tensorflow:Tensor's key in saved_model's tensor_map: input_1
INFO:tensorflow: tensor name: input_1:0, shape: (-1, 224, 224, 3), type: DT_FLOAT
INFO:tensorflow:output tensors info: 
INFO:tensorflow:Tensor's key in saved_model's tensor_map: global_average_pooling2d
INFO:tensorflow: tensor name: global_average_pooling2d/Mean:0, shape: (-1, 1280), type: DT_FLOAT
INF

In [7]:
tflite_model_file = 'MobileNetV2_Model.tflite'
open(tflite_model_file, "wb").write(tflite_model)

8849332

In [8]:
def frame_generator():
    video_paths = tf.io.gfile.glob(VIDEOS_PATH)
    np.random.shuffle(video_paths)
    for video_path in video_paths:
        frames = []
        cap = cv2.VideoCapture(video_path)
        num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        sample_every_frame = max(1, num_frames // SEQUENCE_LENGTH)
        current_frame = 0
        
        label = os.path.basename(os.path.dirname(video_path))
        
        max_images = SEQUENCE_LENGTH
        while True:
            success, frame = cap.read()
            if not success:
                break
                
            if current_frame % sample_every_frame == 0:
                frame = frame[:, :, ::-1]
                img = tf.image.resize(frame, (224, 224))
                img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
                max_images -= 1
                yield img, video_path
                
            if max_images == 0:
                break
            current_frame += 1

In [26]:
# base path of video dataset
BASE_PATH = 'C:\\Users\\STSC\\Desktop\\Project3\\Training_Set\\Full_Data'
VIDEOS_PATH = os.path.join(BASE_PATH, '**', '*.mp4')

# sequence length LSTM will process
SEQUENCE_LENGTH = 40
BATCH_SIZE = 16

In [27]:
dataset = tf.data.Dataset.from_generator(frame_generator,
                                         output_types=(tf.float32, tf.string),
                                         output_shapes=((224, 224, 3), ()))

dataset = dataset.batch(BATCH_SIZE,drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)

In [11]:
current_path = None
all_features = []

# go through the dataset and use the mobilenet_v2 model to 
# extract the features for each frame
for img, batch_paths in tqdm.tqdm(dataset):
    batch_features = feature_extraction_model(img)
    # reshape the tensor
    batch_features = tf.reshape(batch_features,
                                (batch_features.shape[0], -1))
    
    for features, path in zip(batch_features.numpy(), batch_paths.numpy()):
        if path != current_path and current_path is not None:
            output_path = current_path.decode().replace('.mp4','.npy')
            np.save(output_path, all_features)
            all_features = []
            
        current_path = path
        all_features.append(features)

6093it [1:55:09,  1.13s/it]


In [28]:
LABELS = ['CellLeader','ColumnFormation','Come','CoverThisArea','Crouch','Door','Eight','FileFormation','Five','Four','Freeze','Gas','Hostage','HurryUp','IDontUnderstand','IUnderstand','LineAbreastFormation','Listen','Me','Nine','One','Pistol','PointOfEntry','RallyPoint','Rifle','Seven','Shotgun','Six','Sniper','Stop','Ten','Three','Two','Vehicle','Watch','Window','You']
encoder = LabelBinarizer()
encoder.fit(LABELS)

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [29]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=0)


In [30]:
callbacks=[tensorboard_callback]


In [31]:
#setup a keras Sequential model with 1) Masking layer  2) LSTM layer with 512 cells, dropout 0.5, recurrent_dropout of 0.5  
# 3) a fully connected relu activation layer with 256 outputs,  4) a droupout layer 5) a final decision fully connected layer of length of labels
# (which is the number of classes) with softmax activation
model = tf.keras.Sequential([
    tf.keras.layers.Masking(mask_value=0.),
    tf.keras.layers.LSTM(512, dropout=0.5, recurrent_dropout=0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(LABELS), activation='softmax')
])

In [32]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy', 'top_k_categorical_accuracy'])

In [33]:
test_file = 'C:\\Users\\STSC\\Desktop\\Project3\\testfile_01.txt'
train_file = 'C:\\Users\\STSC\\Desktop\\Project3\\trainfile_01.txt'

with open(test_file) as f:
    test_list = [row.strip() for row in list(f)]

with open(train_file) as f:
    train_list = [row.strip() for row in list(f)]
    train_list = [row.split(' ')[0] for row in train_list]


def make_generator(file_list):
    def generator():
        np.random.shuffle(file_list)
        for path in file_list:
            full_path = os.path.join(path).replace('.mp4','.npy')

            label = os.path.basename(os.path.dirname(path))
            features = np.load(full_path, allow_pickle=True) 
            padded_sequence = np.zeros((SEQUENCE_LENGTH, 1280))
            padded_sequence[0:len(features)] = np.array(features)

            transformed_label = encoder.transform([label])
            yield padded_sequence, transformed_label[0]
    return generator

In [39]:
print(test_list)

['C:\\Users\\STSC\\Desktop\\Project3\\Training_Set\\Full_Data\\CellLeader\\R_CellLeader1.mp4', 'C:\\Users\\STSC\\Desktop\\Project3\\Training_Set\\Full_Data\\CellLeader\\R_CellLeader10.mp4', 'C:\\Users\\STSC\\Desktop\\Project3\\Training_Set\\Full_Data\\CellLeader\\R_CellLeader11.mp4', 'C:\\Users\\STSC\\Desktop\\Project3\\Training_Set\\Full_Data\\CellLeader\\R_CellLeader12.mp4', 'C:\\Users\\STSC\\Desktop\\Project3\\Training_Set\\Full_Data\\CellLeader\\R_CellLeader13.mp4', 'C:\\Users\\STSC\\Desktop\\Project3\\Training_Set\\Full_Data\\CellLeader\\R_CellLeader14.mp4', 'C:\\Users\\STSC\\Desktop\\Project3\\Training_Set\\Full_Data\\CellLeader\\R_CellLeader15.mp4', 'C:\\Users\\STSC\\Desktop\\Project3\\Training_Set\\Full_Data\\CellLeader\\R_CellLeader2.mp4', 'C:\\Users\\STSC\\Desktop\\Project3\\Training_Set\\Full_Data\\CellLeader\\R_CellLeader3.mp4', 'C:\\Users\\STSC\\Desktop\\Project3\\Training_Set\\Full_Data\\CellLeader\\R_CellLeader4.mp4', 'C:\\Users\\STSC\\Desktop\\Project3\\Training_Set\\Fu

In [40]:
print(train_list)

['C:\\Users\\STSC\\Desktop\\Project3\\Training_Set\\Full_Data\\Crouch\\Snapchat-1652455056.mp4', 'C:\\Users\\STSC\\Desktop\\Project3\\Training_Set\\Full_Data\\Ten\\VID_20191119_200506.mp4', 'C:\\Users\\STSC\\Desktop\\Project3\\Training_Set\\Full_Data\\CoverThisArea\\H_CoverThisArea6.mp4', 'C:\\Users\\STSC\\Desktop\\Project3\\Training_Set\\Full_Data\\Eight\\H_Eight8.mp4', 'C:\\Users\\STSC\\Desktop\\Project3\\Training_Set\\Full_Data\\You\\VID_20191119_191019.mp4', 'C:\\Users\\STSC\\Desktop\\Project3\\Training_Set\\Full_Data\\Seven\\VID_20191120_234019742.mp4', 'C:\\Users\\STSC\\Desktop\\Project3\\Training_Set\\Full_Data\\Watch\\VID_20191119_192334.mp4', 'C:\\Users\\STSC\\Desktop\\Project3\\Training_Set\\Full_Data\\Crouch\\G_Crouch2.mp4', 'C:\\Users\\STSC\\Desktop\\Project3\\Training_Set\\Full_Data\\RallyPoint\\G_RallyPoint4.mp4', 'C:\\Users\\STSC\\Desktop\\Project3\\Training_Set\\Full_Data\\Door\\H_Door3.mp4', 'C:\\Users\\STSC\\Desktop\\Project3\\Training_Set\\Full_Data\\Come\\VID_201911

In [41]:
# Setup the train_dataset and valid_dataset (validation/testing).  
# Here we setting up training batch sets of 16.  

train_dataset = tf.data.Dataset.from_generator(make_generator(train_list),
                 output_types=(tf.float32, tf.int16),
                 output_shapes=(tf.TensorShape([SEQUENCE_LENGTH, 1280]), tf.TensorShape([len(LABELS)])))
train_dataset = train_dataset.batch(16,drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)


valid_dataset = tf.data.Dataset.from_generator(make_generator(test_list),
                 output_types=(tf.float32, tf.int16),
                 output_shapes=(tf.TensorShape([SEQUENCE_LENGTH, 1280]), tf.TensorShape([len(LABELS)])))
valid_dataset = valid_dataset.batch(16,drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)

In [42]:
print(train_dataset)

<DatasetV1Adapter shapes: ((16, 40, 1280), (16, 37)), types: (tf.float32, tf.int16)>


In [44]:
# fit the model with the training data (can increase the # of epochs & validation_steps if desired)
model.fit(train_dataset, epochs=100, validation_data=valid_dataset, validation_steps=4)

Epoch 1/100


    104/Unknown - 1s 1s/step - loss: 3.2997 - acc: 0.0625 - top_k_categorical_accuracy: 0.43 - 2s 1s/step - loss: 3.2993 - acc: 0.0625 - top_k_categorical_accuracy: 0.37 - 3s 1s/step - loss: 3.3334 - acc: 0.0625 - top_k_categorical_accuracy: 0.35 - 5s 1s/step - loss: 3.3129 - acc: 0.0469 - top_k_categorical_accuracy: 0.37 - 6s 1s/step - loss: 3.3104 - acc: 0.0750 - top_k_categorical_accuracy: 0.35 - 7s 1s/step - loss: 3.3500 - acc: 0.0625 - top_k_categorical_accuracy: 0.31 - 8s 1s/step - loss: 3.3068 - acc: 0.0625 - top_k_categorical_accuracy: 0.33 - 9s 1s/step - loss: 3.3133 - acc: 0.0547 - top_k_categorical_accuracy: 0.34 - 10s 1s/step - loss: 3.3099 - acc: 0.0625 - top_k_categorical_accuracy: 0.347 - 11s 1s/step - loss: 3.2890 - acc: 0.0688 - top_k_categorical_accuracy: 0.350 - 13s 1s/step - loss: 3.3018 - acc: 0.0625 - top_k_categorical_accuracy: 0.346 - 14s 1s/step - loss: 3.3084 - acc: 0.0625 - top_k_categorical_accuracy: 0.338 - 15s 1s/step - loss: 3.3133 - acc: 0.0673 - top_k_c

180/180 [==============================].2078 - acc: 0.0893 - top_k_categorical_accuracy: 0.36 - 122s 1s/step - loss: 3.2094 - acc: 0.0890 - top_k_categorical_accuracy: 0.36 - 123s 1s/step - loss: 3.2085 - acc: 0.0894 - top_k_categorical_accuracy: 0.36 - 124s 1s/step - loss: 3.2090 - acc: 0.0897 - top_k_categorical_accuracy: 0.36 - 126s 1s/step - loss: 3.2081 - acc: 0.0889 - top_k_categorical_accuracy: 0.36 - 127s 1s/step - loss: 3.2071 - acc: 0.0892 - top_k_categorical_accuracy: 0.36 - 128s 1s/step - loss: 3.2068 - acc: 0.0890 - top_k_categorical_accuracy: 0.36 - 129s 1s/step - loss: 3.2060 - acc: 0.0882 - top_k_categorical_accuracy: 0.36 - 131s 1s/step - loss: 3.2054 - acc: 0.0885 - top_k_categorical_accuracy: 0.36 - 132s 1s/step - loss: 3.2040 - acc: 0.0894 - top_k_categorical_accuracy: 0.36 - 133s 1s/step - loss: 3.2037 - acc: 0.0891 - top_k_categorical_accuracy: 0.36 - 134s 1s/step - loss: 3.2011 - acc: 0.0905 - top_k_categorical_accuracy: 0.36 - 135s 1s/step - loss: 3.1982 - acc:

108/180 [=================>............] - ETA: 4:18 - loss: 3.1636 - acc: 0.0625 - top_k_categorical_accuracy: 0.50 - ETA: 4:09 - loss: 3.0739 - acc: 0.0625 - top_k_categorical_accuracy: 0.56 - ETA: 4:03 - loss: 3.0556 - acc: 0.0833 - top_k_categorical_accuracy: 0.50 - ETA: 4:00 - loss: 3.0759 - acc: 0.0938 - top_k_categorical_accuracy: 0.45 - ETA: 4:04 - loss: 3.0379 - acc: 0.0750 - top_k_categorical_accuracy: 0.50 - ETA: 4:00 - loss: 3.0442 - acc: 0.0729 - top_k_categorical_accuracy: 0.47 - ETA: 3:59 - loss: 2.9872 - acc: 0.0893 - top_k_categorical_accuracy: 0.50 - ETA: 4:01 - loss: 2.9841 - acc: 0.1016 - top_k_categorical_accuracy: 0.50 - ETA: 3:57 - loss: 2.9716 - acc: 0.0972 - top_k_categorical_accuracy: 0.50 - ETA: 3:53 - loss: 2.9389 - acc: 0.1063 - top_k_categorical_accuracy: 0.51 - ETA: 3:49 - loss: 2.9476 - acc: 0.1080 - top_k_categorical_accuracy: 0.50 - ETA: 3:45 - loss: 2.9615 - acc: 0.1094 - top_k_categorical_accuracy: 0.50 - ETA: 3:43 - loss: 2.9376 - acc: 0.1202 - top_

180/180 [==============================] - ETA: 1:43 - loss: 2.8201 - acc: 0.1829 - top_k_categorical_accuracy: 0.51 - ETA: 1:41 - loss: 2.8192 - acc: 0.1824 - top_k_categorical_accuracy: 0.51 - ETA: 1:40 - loss: 2.8190 - acc: 0.1819 - top_k_categorical_accuracy: 0.51 - ETA: 1:38 - loss: 2.8189 - acc: 0.1819 - top_k_categorical_accuracy: 0.51 - ETA: 1:37 - loss: 2.8202 - acc: 0.1820 - top_k_categorical_accuracy: 0.51 - ETA: 1:36 - loss: 2.8177 - acc: 0.1837 - top_k_categorical_accuracy: 0.51 - ETA: 1:34 - loss: 2.8180 - acc: 0.1821 - top_k_categorical_accuracy: 0.51 - ETA: 1:33 - loss: 2.8211 - acc: 0.1810 - top_k_categorical_accuracy: 0.51 - ETA: 1:31 - loss: 2.8203 - acc: 0.1811 - top_k_categorical_accuracy: 0.51 - ETA: 1:30 - loss: 2.8194 - acc: 0.1801 - top_k_categorical_accuracy: 0.51 - ETA: 1:28 - loss: 2.8213 - acc: 0.1796 - top_k_categorical_accuracy: 0.51 - ETA: 1:27 - loss: 2.8207 - acc: 0.1807 - top_k_categorical_accuracy: 0.51 - ETA: 1:26 - loss: 2.8206 - acc: 0.1808 - top_

...

180/180 [==============================] - ETA: 1:24 - loss: 0.5483 - acc: 0.8526 - top_k_categorical_accuracy: 0.98 - ETA: 1:23 - loss: 0.5456 - acc: 0.8534 - top_k_categorical_accuracy: 0.98 - ETA: 1:22 - loss: 0.5433 - acc: 0.8542 - top_k_categorical_accuracy: 0.98 - ETA: 1:21 - loss: 0.5446 - acc: 0.8538 - top_k_categorical_accuracy: 0.98 - ETA: 1:20 - loss: 0.5415 - acc: 0.8540 - top_k_categorical_accuracy: 0.98 - ETA: 1:18 - loss: 0.5404 - acc: 0.8547 - top_k_categorical_accuracy: 0.98 - ETA: 1:17 - loss: 0.5376 - acc: 0.8549 - top_k_categorical_accuracy: 0.98 - ETA: 1:16 - loss: 0.5360 - acc: 0.8551 - top_k_categorical_accuracy: 0.98 - ETA: 1:15 - loss: 0.5341 - acc: 0.8552 - top_k_categorical_accuracy: 0.98 - ETA: 1:14 - loss: 0.5377 - acc: 0.8549 - top_k_categorical_accuracy: 0.98 - ETA: 1:12 - loss: 0.5366 - acc: 0.8550 - top_k_categorical_accuracy: 0.98 - ETA: 1:11 - loss: 0.5350 - acc: 0.8552 - top_k_categorical_accuracy: 0.98 - ETA: 1:10 - loss: 0.5343 - acc: 0.8554 - top_

108/180 [=================>............] - ETA: 3:29 - loss: 0.7458 - acc: 0.8125 - top_k_categorical_accuracy: 1.00 - ETA: 3:26 - loss: 0.6065 - acc: 0.8438 - top_k_categorical_accuracy: 1.00 - ETA: 3:24 - loss: 0.5512 - acc: 0.8542 - top_k_categorical_accuracy: 1.00 - ETA: 3:22 - loss: 0.4566 - acc: 0.8750 - top_k_categorical_accuracy: 1.00 - ETA: 3:21 - loss: 0.4179 - acc: 0.8750 - top_k_categorical_accuracy: 1.00 - ETA: 3:20 - loss: 0.3944 - acc: 0.8750 - top_k_categorical_accuracy: 1.00 - ETA: 3:19 - loss: 0.4247 - acc: 0.8571 - top_k_categorical_accuracy: 1.00 - ETA: 3:18 - loss: 0.4472 - acc: 0.8516 - top_k_categorical_accuracy: 1.00 - ETA: 3:18 - loss: 0.4800 - acc: 0.8333 - top_k_categorical_accuracy: 1.00 - ETA: 3:17 - loss: 0.4778 - acc: 0.8313 - top_k_categorical_accuracy: 1.00 - ETA: 3:16 - loss: 0.5013 - acc: 0.8239 - top_k_categorical_accuracy: 1.00 - ETA: 3:15 - loss: 0.5466 - acc: 0.8073 - top_k_categorical_accuracy: 0.99 - ETA: 3:14 - loss: 0.5245 - acc: 0.8173 - top_

180/180 [==============================] - ETA: 1:24 - loss: 0.5547 - acc: 0.8492 - top_k_categorical_accuracy: 0.99 - ETA: 1:22 - loss: 0.5516 - acc: 0.8494 - top_k_categorical_accuracy: 0.99 - ETA: 1:21 - loss: 0.5519 - acc: 0.8497 - top_k_categorical_accuracy: 0.99 - ETA: 1:20 - loss: 0.5518 - acc: 0.8493 - top_k_categorical_accuracy: 0.99 - ETA: 1:19 - loss: 0.5495 - acc: 0.8496 - top_k_categorical_accuracy: 0.99 - ETA: 1:18 - loss: 0.5488 - acc: 0.8498 - top_k_categorical_accuracy: 0.99 - ETA: 1:17 - loss: 0.5486 - acc: 0.8495 - top_k_categorical_accuracy: 0.99 - ETA: 1:15 - loss: 0.5459 - acc: 0.8497 - top_k_categorical_accuracy: 0.99 - ETA: 1:14 - loss: 0.5421 - acc: 0.8504 - top_k_categorical_accuracy: 0.99 - ETA: 1:13 - loss: 0.5436 - acc: 0.8501 - top_k_categorical_accuracy: 0.99 - ETA: 1:12 - loss: 0.5407 - acc: 0.8508 - top_k_categorical_accuracy: 0.99 - ETA: 1:10 - loss: 0.5408 - acc: 0.8505 - top_k_categorical_accuracy: 0.99 - ETA: 1:09 - loss: 0.5431 - acc: 0.8502 - top_

108/180 [=================>............] - ETA: 3:30 - loss: 0.9518 - acc: 0.8125 - top_k_categorical_accuracy: 0.93 - ETA: 3:26 - loss: 0.6386 - acc: 0.8438 - top_k_categorical_accuracy: 0.96 - ETA: 3:24 - loss: 0.5585 - acc: 0.8333 - top_k_categorical_accuracy: 0.97 - ETA: 3:23 - loss: 0.4998 - acc: 0.8281 - top_k_categorical_accuracy: 0.98 - ETA: 3:22 - loss: 0.4233 - acc: 0.8625 - top_k_categorical_accuracy: 0.98 - ETA: 3:21 - loss: 0.4260 - acc: 0.8542 - top_k_categorical_accuracy: 0.97 - ETA: 3:20 - loss: 0.4674 - acc: 0.8482 - top_k_categorical_accuracy: 0.97 - ETA: 3:18 - loss: 0.4847 - acc: 0.8594 - top_k_categorical_accuracy: 0.97 - ETA: 3:17 - loss: 0.4516 - acc: 0.8611 - top_k_categorical_accuracy: 0.97 - ETA: 3:16 - loss: 0.4769 - acc: 0.8562 - top_k_categorical_accuracy: 0.98 - ETA: 3:14 - loss: 0.4744 - acc: 0.8466 - top_k_categorical_accuracy: 0.98 - ETA: 3:13 - loss: 0.4519 - acc: 0.8490 - top_k_categorical_accuracy: 0.98 - ETA: 3:12 - loss: 0.4384 - acc: 0.8510 - top_

180/180 [==============================] - ETA: 1:21 - loss: 0.6436 - acc: 0.8394 - top_k_categorical_accuracy: 0.98 - ETA: 1:20 - loss: 0.6446 - acc: 0.8386 - top_k_categorical_accuracy: 0.98 - ETA: 1:19 - loss: 0.6448 - acc: 0.8378 - top_k_categorical_accuracy: 0.98 - ETA: 1:18 - loss: 0.6482 - acc: 0.8371 - top_k_categorical_accuracy: 0.98 - ETA: 1:17 - loss: 0.6514 - acc: 0.8368 - top_k_categorical_accuracy: 0.98 - ETA: 1:16 - loss: 0.6476 - acc: 0.8372 - top_k_categorical_accuracy: 0.98 - ETA: 1:15 - loss: 0.6499 - acc: 0.8364 - top_k_categorical_accuracy: 0.98 - ETA: 1:13 - loss: 0.6467 - acc: 0.8367 - top_k_categorical_accuracy: 0.98 - ETA: 1:12 - loss: 0.6485 - acc: 0.8376 - top_k_categorical_accuracy: 0.98 - ETA: 1:11 - loss: 0.6511 - acc: 0.8369 - top_k_categorical_accuracy: 0.98 - ETA: 1:10 - loss: 0.6487 - acc: 0.8372 - top_k_categorical_accuracy: 0.98 - ETA: 1:09 - loss: 0.6530 - acc: 0.8359 - top_k_categorical_accuracy: 0.98 - ETA: 1:08 - loss: 0.6486 - acc: 0.8368 - top_

In [46]:
# save model as a HDF5 file
BASE_DIRECTORY = 'C:\\Users\\STSC\\Desktop\\Project3';
model_file = os.path.join(BASE_DIRECTORY, 'TF1-14_LSTM_Model_V1.h5')
model.save(model_file)

In [61]:
# save model in SavedModel format for serving from the cloud
SAVED_MODEL_DIR = os.path.join(BASE_DIRECTORY, 'SavedModelDir/LSTM/V1/')
tf.keras.experimental.export_saved_model(model, SAVED_MODEL_DIR)

AssertionError: Export directory already exists, and isn't empty. Please choose a different export directory, or delete all the contents of the specified directory: C:\Users\STSC\Desktop\Project3\SavedModelDir/LSTM/V1/

In [62]:
# Load TensorBoard notebook extension
%reload_ext tensorboard
%tensorboard --logdir="logs" --host localhost --port=8080

Reusing TensorBoard on port 8080 (pid 2192), started 2:49:29 ago. (Use '!kill 2192' to kill it.)

In [59]:
tensorboard --help.

ERROR: Failed to launch TensorBoard (exited with 2).
Contents of stderr:
c:\programdata\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\programdata\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\programdata\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\programda